In [100]:
import numpy as np
import pandas as pd
from sklearn import datasets 
from sklearn import metrics
from sklearn import model_selection
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def mse_error(y, X, w):
    return ((y - linear_prediction(X, w)) ** 2).mean()

In [3]:
def linear_prediction(X, w):
    return X.dot(w)

In [4]:
def gradient(X, w, y):
    return (2/y.shape[0]) * (X.T.dot((y - linear_prediction(X, w))))

In [5]:
def gradient_stochastic(X, w, y):
    return 2 * (X.T.dot((y - linear_prediction(X, w))))

In [6]:
data, target, coef = datasets.make_regression(n_features = 2, n_informative = 1, n_targets = 1, 
                                              noise = 5., coef = True, random_state = 2)

In [94]:
datasets.make_regression?

In [7]:
def gradient_desent(X, y, w_init, max_iter, weight_dist, step):
    iter_ = 0
    w = w_init
    weight_dist = np.inf
    errors = []
    
    while (iter_ < max_iter) or (dist > weight_dist):
        w_new = w + step * gradient(X, w, y)
        dist = np.linalg.norm(w - w_new)
        w = w_new
        errors.append(mse_error(y, X, w))
        iter_ += 1
        
    return w, errors

In [8]:
def stochastic_gradient_desent(X, y, w_init, max_iter, weight_dist, step):
    iter_ = 0
    w = w_init
    weight_dist = np.inf
    errors = []

    while (iter_ < max_iter) or (dist > weight_dist):
        index = np.random.randint(X.shape[0], size=1)[0]
        w_new = w + step * gradient_stochastic(X[index], w, y[index])
        dist = np.linalg.norm(w - w_new)
        w = w_new
        errors.append(mse_error(y[index], X[index], w))
        iter_ += 1
        
    return w, errors

In [9]:
%%time 
batch = gradient_desent(data, target, w_init=[0,0], max_iter=1e5, weight_dist=1e-15, step=1e-4)

CPU times: user 4.05 s, sys: 8 ms, total: 4.06 s
Wall time: 4.05 s


In [10]:
%%time
stochastic = stochastic_gradient_desent(data, target, w_init=[0,0], max_iter=1e5, weight_dist=1e-15, step=1e-4)

CPU times: user 4.52 s, sys: 20 ms, total: 4.54 s
Wall time: 4.54 s


In [85]:
class LinearRegression():
    def __init__(self, alpha=None, regularization=None, max_iter=1e5, weight_dist=1e-15, step=1e-4):
        self.regularization = None 
        self.alpha = None
        self.errors = []
        self.w = []
        self.max_iter = max_iter
        self.weight_dist = weight_dist
        self.step = step
        
    def predict(self, X):
        X = np.hstack((np.ones((X.shape[0], 1), dtype=X.dtype), X))
        y = np.array([sum(i*self.w) for i in X])
        return y
    
    def linear_prediction(self, X, w):
        return X.dot(w)
    
    def gradient_stochastic(self, X, w, y):
        return 2 * (X.T.dot((y - self.linear_prediction(X, w))))
    
    
    def mse_error(self, y, X, w):
        return ((y - self.linear_prediction(X, w)) ** 2).mean()
        
        
    def fit(self, X, y):
        X = np.hstack((np.ones((X.shape[0], 1), dtype=X.dtype), X))
        if self.regularization is None:
            self.stochastic_gradient_desent(X, y, np.zeros(X.shape[1]), max_iter=self.max_iter, \
                                            weight_dist=self.weight_dist, step=self.step)
        else:
            pass
#             if regularization is 'lasso':
#                 self.stochastic_gradient_desent(X, y, np.zeros(X.shape[1]), max_iter=self.max_iter, \
#                             weight_dist=self.weight_dist, step=self.step)
#             else:
#                 self.stochastic_gradient_desent(X, y, np.zeros(X.shape[1]), max_iter=self.max_iter, \
#                             weight_dist=self.weight_dist, step=self.step)
            
    def stochastic_gradient_desent(self, X, y, w_init, max_iter, weight_dist, step):
        iter_ = 0
        w = w_init
        weight_dist = np.inf
        errors = []

        while (iter_ < max_iter) or (dist > weight_dist):
            index = np.random.randint(X.shape[0], size=1)[0]
            w_new = w + step * self.gradient_stochastic(X[index], w, y[index])
            dist = np.linalg.norm(w - w_new)
            w = w_new
            errors.append(self.mse_error(y[index], X[index], w))
            iter_ += 1
            
        self.w = w
        self.erros = errors
    